# 一、文档拆分器 Text Splitters

## 1、为什么要拆分/分块？

- 当获取到统一的Document对象后,需要切勿分成Chunk块,如果不切分,而考虑将其作为一个整体的Document对象,会存在问题如下:
    - 1）如果用户问题所需的答案出现在某一个Document对象中,则将检索到的整个Document对象直接放入Prompt中并不是最优选择,其中会包含较多无关信息,无效信息越多,对大模型后续推理的影响越大
    - 2）任何大模型都存在最大输入的Token限制,如果一个Document非常大,那大模型可能会无法容纳/理解这么多信息

## 2、Chunking的拆分策略

- 策略1: 根据句子切分 =》按照自然句子边界(分隔符为句号)进行切分,以保持语义完整性
- 策略2: 按照固定字符数来切分 =》根据特定的字符数来划分文本,不考虑文本的实际内容和语义,可能会造成主题或语义上断裂
- 策略3: 按照固定字符数/Token数来切分,并结合重叠窗口 =》在按字符数切分的基础上,通过重叠窗口技术 以避免切分关键内容,确保信息的连贯性
- 策略4: 递归字符切分方法 =》通过递归字符 动态确定切分点,可根据文档的复杂性和内容密度来调整块的大小,高效且灵活,结合了固定长度切分和语义分析(可确保每个段落包含一个完整的主题),为首选策略
- 策略5: 根据语义内容切分 =》依据文本的语义内容来划分文本块,以保持相关信息的集中和完整,适用于需要保持 高度语义的应用场景
        效率较低,需要运行复杂的分段算法,处理速度较慢,且段落长度极不均匀

## 3、代码实践

### 1）CharacterTextSplitter: 按字符数分块

- 参数情况说明：
    - chunk_size: 每个切块的最大token数量，默认值为4000
    - chunk_overlap: 相邻两个切块之间的最大重叠token数量，默认值为200
    - separator: 分割使用的分隔符，默认值为"\n\n"。
    - length_function: 用于计算切块长度的方法。默认赋值为父类TextSplitter的len函数
- 若必须禁用分隔符（如处理无空格文本），需容忍实际块长略小于 chunk_size （尤其对中文）

In [20]:
from langchain_text_splitters import CharacterTextSplitter

# 构造一段较长的文本，需要体现分割和重叠的效果
text = (
    "LangChain是一个用于开发支持连接到语言模型的应用程序的框架。"
    "它帮助开发者更轻松地将语言模型与外部数据、API和用户输入结合。"
    "当面对较长的文档时，通常需要将文档拆分为更小的部分，"
    "以便于下游处理，例如向量检索或问答系统。"
    "使用CharacterTextSplitter可以灵活地控制分块的大小和重叠，"
    "确保上下文衔接，减少信息丢失。"
)

# 创建字符分割器
splitter = CharacterTextSplitter(
    chunk_size=50,   # 每个分块的大小为40个字符
    chunk_overlap=5, # 每个分块之间有10个字符的重叠
    length_function=len,   # 计算文本块长度的函数为len
    separator="",    # 设置空字符串时,表示优先禁用分隔符，按字符计数
)

# 分割文本
texts = splitter.split_text(text)

for i, chunk in enumerate(texts):
    print(f"分块 {i+1}, 长度：{len(chunk)}")
    print(chunk)
    print("=" * 20)

分块 1, 长度：50
LangChain是一个用于开发支持连接到语言模型的应用程序的框架。它帮助开发者更轻松地将语言模型与
分块 2, 长度：50
语言模型与外部数据、API和用户输入结合。当面对较长的文档时，通常需要将文档拆分为更小的部分，以便于
分块 3, 长度：50
分，以便于下游处理，例如向量检索或问答系统。使用CharacterTextSplitter可以灵活地
分块 4, 长度：31
可以灵活地控制分块的大小和重叠，确保上下文衔接，减少信息丢失。


- separator优先原则：当设置了 separator （如"。"），分割器会首先尝试在句号分隔符("。")处分割，然后再考虑 chunk_size。这是为了避免在句子中间硬性切断。
这种设计是为了:
    - 1）优先保持语义完整性（不切断句子）
    - 2）避免产生无意义的碎片（如半个单词/不完整句子）
    - 3）如果 chunk_size 比片段小，无法拆分片段，导致 overlap失效
    - 4）chunk_overlap仅在合并后的片段之间生效（如果 chunk_size 足够大）。如果没有合并的片段，则 overlap失效
- - 指明使用句号作为分隔符的代码示例如下:

In [21]:
splitter = CharacterTextSplitter(
    chunk_size=50,   # 每个分块的大小为40个字符
    chunk_overlap=5, # 每个分块之间有10个字符的重叠
    length_function=len,   # 计算文本块长度的函数为len
    separator="。",    # 按照句号分割
)

# 分割文本
texts = splitter.split_text(text)

for i, chunk in enumerate(texts):
    print(f"分块 {i+1}, 长度：{len(chunk)}")
    print(chunk)
    print("=" * 20)

分块 1, 长度：33
LangChain是一个用于开发支持连接到语言模型的应用程序的框架
分块 2, 长度：31
它帮助开发者更轻松地将语言模型与外部数据、API和用户输入结合
分块 3, 长度：45
当面对较长的文档时，通常需要将文档拆分为更小的部分，以便于下游处理，例如向量检索或问答系统
分块 4, 长度：53
使用CharacterTextSplitter可以灵活地控制分块的大小和重叠，确保上下文衔接，减少信息丢失


- chunk_overlap重叠部分说明:

In [27]:
text = "说明chunk_overlap的作用,这是第一段文本内容。这是第二段文本内容。这是第三段文本内容。最后一段文本内容。"

splitter = CharacterTextSplitter(
    chunk_size=10,
    chunk_overlap=5,
    separator="。",
    keep_separator=True
)
texts = splitter.split_text(text)

print(f"原始文本: {text}")

for i, chunk in enumerate(texts):
    print(f"分块 {i+1}, 长度: {len(chunk)}")
    print(chunk)
    print("=" * 20)

Created a chunk of size 28, which is longer than the specified 10


原始文本: 说明chunk_overlap的作用,这是第一段文本内容。这是第二段文本内容。这是第三段文本内容。最后一段文本内容。
分块 1, 长度: 28
说明chunk_overlap的作用,这是第一段文本内容
分块 2, 长度: 10
。这是第二段文本内容
分块 3, 长度: 10
。这是第三段文本内容
分块 4, 长度: 10
。最后一段文本内容。


- 如上, 当设置了 separator （如"。"），分割器会首先尝试在句号分隔符("。")处分割，然后再考虑 chunk_size

### 2）RecursiveCharacterTextSplitter: 按递归字符分块

- 文档切分器中较常用的是 RecursiveCharacterTextSplitter (递归字符文本切分器) ，遇到 特定字符 时进行分割。默认情况下，它尝试进行切割的字符包括 ["\n\n", "\n", " ", ""]
- 具体为: 根据第一个字符进行切块，但如果任何切块太大，则会继续移动到下一个字符继续切块，以此类推。
此外，还可以考虑添加，。等分割字符。
- 特点:
    - 保留上下文: 优先在自然语言边界（如段落、句子结尾）处分割， 减少信息碎片化
    - 智能分段: 通过递归尝试多种分隔符，将文本分割为大小接近chunk_size的片段
    - 灵活适配: 适用于多种文本类型(代码、Markdown、普通文本等)，是LangChain中最通用的文本拆分器
此外，还可以指定的参数包括:
    - chunk_size、chunk_overlap、length_function、add_start_index: 同TextSplitter（父类）

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 准备一个包含多种分割符、有自然语义边界的示例文本，方便演示递归分割器的智能分段
text = (
    "RecursiveCharacterTextSplitter演示：\n\n"
    "这是第一段文本。\n"
    "这一行内容较短。\n"
    "下面是第二段内容，带有更多字符和描述，用于观察分割效果。\n"
    "最后一部分，包含一些语义上的分隔符，比如逗号，句号。"
)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,             # 每个分块最大20字符，便于观察边界和递归分割行为
    chunk_overlap=5,           # 分块之间有5字符重叠，更好地保留上下文
    separators=["\n\n", "\n", "。", "，", " "],  # 递归尝试多种分割符，优先在自然语言边界分割
    add_start_index=True       # 展示每个分块在原文中的起始位置
)

# chunks = splitter.split_text(text)

# 使用 create_documents 而不是 split_text。create_documents 返回包含文本与元信息的文档对象列表
chunks = splitter.create_documents([text])

for i, chunk in enumerate(chunks):  # 每个chunk都是 <class 'langchain_core.documents.base.Document'>
    print(f"分块 {i+1}")
    print(chunk)
    print("=" * 20)

分块 1
page_content='RecursiveCharacterTextSplitter演示：' metadata={'start_index': 0}
分块 2
page_content='这是第一段文本。
这一行内容较短。' metadata={'start_index': 35}
分块 3
page_content='下面是第二段内容，带有更多字符和描述' metadata={'start_index': 53}
分块 4
page_content='，用于观察分割效果' metadata={'start_index': 71}
分块 5
page_content='。' metadata={'start_index': 80}
分块 6
page_content='最后一部分，包含一些语义上的分隔符' metadata={'start_index': 82}
分块 7
page_content='，比如逗号，句号' metadata={'start_index': 99}
分块 8
page_content='。' metadata={'start_index': 107}


### 3）TokenTextSplitter: 按Token数分块

- 将大文本拆分为chunk块时,除了字符以外,还可以按Token数来分割(而非字符或单词数),将长文本切分为多个小块
- 对大模型而言,Token是文本的最小处理单元,例如:
    - 英文: "hello" → 1个Token， "ChatGPT" → 2个Token("Chat" + "GPT")
    - 中文: "人工智能" → 可能拆分为2-3个Token(取决于分词器)
- 为什么要按照Token分割?
    - 语言模型 对输入长度的限制是基于Token数的,直接按照字符或单词分割可能会导致实际Token数 超出限度(需确保每个文本块不超过大模型的Token上限)
    - 大模型(LLM)通常是以Token数作为其计量(或收费)的依据,所以采用Token分割也有助于我们在使用时更方便地控制成本

- 特别注意: 字符长度不等于Token数

In [46]:
# 演示 TokenTextSplitter 可按「Token数量」切分文本并控制 Token 重叠
from langchain_text_splitters import TokenTextSplitter

# 示例文本：涵盖英文、中文、符号等多样内容
text = [
    "人工智能（AI）正在改变世界，包括医疗、金融、自动驾驶等领域。",
    "LangChain库可以让我们快速实现基于大语言模型的应用开发。",
    "OpenAI's GPT-4模型能够理解、生成更复杂的语言、代码与知识。",
    "Prompt Engineering是现代AI产品设计的核心能力。"
]

# 以不同参数突出 TokenTextSplitter 对分割方式的影响
splitter = TokenTextSplitter(
    chunk_size=20,           # 每个分块最多包含15个Token
    chunk_overlap=5,         # 每个块与上一个块有3个Token重叠
    encoding_name="cl100k_base",  # (OpenAI模型使用的)tokenizer的名称
    strip_whitespace=True,   # 自动移除分块前后的空白符
    add_start_index=True     # 在元信息中返回原文起始位置，方便trace
)

texts = splitter.create_documents(text)

for i, chunk in enumerate(texts):
    print(f"块 {i+1}, 长度: {len(chunk.page_content)}, 内容: {chunk}")
    print("=" * 50)

块 1, 长度: 18, 内容: page_content='人工智能（AI）正在改变世界，包括医' metadata={'start_index': 0}
块 2, 长度: 12, 内容: page_content='包括医疗、金融、自动驾驶' metadata={'start_index': 15}
块 3, 长度: 7, 内容: page_content='动驾驶等领域。' metadata={'start_index': 24}
块 4, 长度: 27, 内容: page_content='LangChain库可以让我们快速实现基于大语言模型的' metadata={'start_index': 0}
块 5, 长度: 10, 内容: page_content='语言模型的应用开发。' metadata={'start_index': 22}
块 6, 长度: 26, 内容: page_content='OpenAI's GPT-4模型能够理解、生成更复杂' metadata={'start_index': 0}
块 7, 长度: 15, 内容: page_content='生成更复杂的语言、代码与知识。' metadata={'start_index': 21}
块 8, 长度: 33, 内容: page_content='Prompt Engineering是现代AI产品设计的核心能力。' metadata={'start_index': 0}


### 4）SemanticChunker：语义分块

- SemanticChunking(语义分块): 超越了传统的基于字符或固定大小的分块方式,而是根据文本的语义结构进行智能分块,使每个分块保持语义完整性,从而提高检索增强生成(RAG)等应用的效果

- 语义分割 vs 传统分割
| 特性         | 语义分割（SemanticChunker）          | 传统字符分割（RecursiveCharacter） |
|------------|------------------------------|-----------------------------|
| 分割依据      | 嵌入向量相似度                        | 固定字符/换行符                   |
| 语义完整性    | ✅ 保持主题连贯                         | ❌ 可能切断句子逻辑                  |
| 计算成本      | ❌ 高（需嵌入模型）                     | ✅ 低                              |
| 适用场景      | 需要高语义一致性的任务                    | 简单文本预处理                      |

In [60]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
import os
import dotenv

dotenv.load_dotenv()

# 加载文本
with open("document/load/09-embedding-doc.txt", encoding="utf-8") as f:
    state_of_the_union = f.read() #返回字符串

# 获取嵌入模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

embed_model = OpenAIEmbeddings(
    model="text-embedding-3-large"
)
# 获取切割器
text_splitter = SemanticChunker(
    embeddings=embed_model,
    breakpoint_threshold_type="percentile",#断点阈值类型：字面值["百分位数", "标准差", "四分位距", "梯度"] 选其一
    breakpoint_threshold_amount=65.0 #断点阈值数量 (极低阈值 → 高分割敏感度)
)

# 切分文档
docs = text_splitter.create_documents(texts = [state_of_the_union])

print(len(docs))

for doc in docs:
    print(f"🔍 文档 {doc}:")

4
🔍 文档 page_content='人工智能综述：发展、应用与未来展望

摘要
人工智能（Artificial Intelligence，AI）作为计算机科学的一个重要分支，近年来取得了突飞猛进的发展。本文综述了人工智能的发展历程、核心技术、应用领域以及未来发展趋势。通过对人工智能的定义、历史背景、主要技术（如机器学习、深度学习、自然语言处理等）的详细介绍，探讨了人工智能在医疗、金融、教育、交通等领域的应用，并分析了人工智能发展过程中面临的挑战与机遇。最后，本文对人工智能的未来发展进行了展望，提出了可能的突破方向。

1. 引言
人工智能是指通过计算机程序模拟人类智能的一门科学。自20世纪50年代诞生以来，人工智能经历了多次起伏，近年来随着计算能力的提升和大数据的普及，人工智能技术取得了显著的进展。人工智能的应用已经渗透到日常生活的方方面面，从智能手机的语音助手到自动驾驶汽车，从医疗诊断到金融分析，人工智能正在改变着人类社会的运行方式。

2.':
🔍 文档 page_content='人工智能的发展历程
2.1 早期发展
人工智能的概念最早可以追溯到20世纪50年代。1956年，达特茅斯会议（Dartmouth Conference）被认为是人工智能研究的正式开端。在随后的几十年里，人工智能研究经历了多次高潮与低谷。早期的研究主要集中在符号逻辑和专家系统上，但由于计算能力的限制和算法的不足，进展缓慢。
2.2 机器学习的兴起
20世纪90年代，随着统计学习方法的引入，机器学习逐渐成为人工智能研究的主流。支持向量机（SVM）、决策树、随机森林等算法在分类和回归任务中取得了良好的效果。这一时期，机器学习开始应用于数据挖掘、模式识别等领域。
2.3 深度学习的突破
2012年，深度学习在图像识别领域取得了突破性进展，标志着人工智能进入了一个新的阶段。深度学习通过多层神经网络模拟人脑的工作方式，能够自动提取特征并进行复杂的模式识别。卷积神经网络（CNN）、循环神经网络（RNN）和长短期记忆网络（LSTM）等深度学习模型在图像处理、自然语言处理、语音识别等领域取得了显著成果。

3. 人工智能的核心技术
3.1 机器学习
机器学习是人工智能的核心技术之一，通过算法使计算机从数据中学习并做出决策。常见的机器学习算法包括监督学习、无监督学习和强化学习。监督学习通过标记数据

重要参数说明:


- 1）breakpoint_threshold_type(断点阈值类型)
作用: 定义文本语义边界的检测算法,决定何时分割文本块
可选值相关总结如下:

| 类型                 | 原理说明                                     | 适用场景             |
|--------------------|------------------------------------------|------------------|
| percentile         | 计算相邻句子嵌入向量的余弦距离,取距离分布的第N百分位值作为阈值,高于此值则分割 | 常规文本(如文章,报告)     |
| standard_deviation | 以均值 + N倍标准差为阈值，识别语义突变点                   | 语义变化剧烈的文档(如技术手册) |
| interquartile      | 用四分位距（IQR） 定义异常值边界，超过则分割                 | 长文档(如书籍)         |
| gradient           | 基于嵌入向量变化的梯度检测分割点(需自定义实现)                 | 实验性需求            |

- 2）breakpoint_threshold_amount(断点阈值量)
    - 作用: 控制分割的粒度敏感度，值越小分割越细（块越多），值越大分割越粗（块越少）。
    - 取值范围与示例:
        - percentile 模式：0.0~100.0，用户代码设 65.0 表示仅当余弦距离 > 所有距离中最低的65.0%值时分割 。默认值是：95.0，兼顾语义完整性与检索效率。
                值过小（比如0.1），会产生大量小文本块，过度分割可能导致上下文断裂。
        - standard_deviation 模式：浮点数（如 1.5 表示均值+1.5倍标准差）。
        - interquartile 模式：倍数（如 1.5 是IQR标准值）。